In [1]:
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.4.1


In [2]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('geonrw-patches') # you can list the bucket with "!gsutil ls $GCS_DS_PATH"
GCS_DS_PATH

'gs://kds-2da24647f44cced3d49838d7c8b0c02ed79a33d3ad04ad70aa543eac'

In [3]:
RESUNETA_SAVED_PATH = KaggleDatasets().get_gcs_path('resuneta-trained-p256')
RESUNETA_SAVED_PATH

'gs://kds-0c3997d4b61e0c01305ad79b43d9d5777050be683ba836df79d562f7'

In [4]:
!pip install keras_unet_collection tensorflow-addons

     |████████████████████████████████| 67 kB 607 kB/s eta 0:00:01


In [5]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code

try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  1


In [17]:
import sys

import numpy as np
from matplotlib import pyplot as plt

# use previous weights and continue from there, or restart and delete previous state
CONTINUE_LEARNING = True

N_EPOCHS = 250
USE_ELEVATION = True

PATCH_SIZE = 256  # should be divisible by 32 (based on the models used)
TRAIN_RATIO = 0.85  # ratio of inputs used for training, out of all the inputs (=1)
VALIDATION_RATIO = 0.08

BASIC_BATCH_SIZE = 2
UNET_BATCH_MULTIPLIER = 4  # U-Net uses less RAM than ResUNet-a, can handle more patches at once


# - if we want to hasten the training (or evaluating) process, we can specify a number
#    to divide the amount of available inputs (for a specific purpose in question) with.
DATA_DIVIDER = 1.  # this way there will be (16 |) 48 training tfrecords used
    # (good for TPU). (If TRAIN_RATIO stays 0.8)

USED_IMG_COUNT = 7356  # out of 7356 images. lower it to save disk space with patch generation

GROUND_TRUTH_SHAPE = [PATCH_SIZE, PATCH_SIZE, 1]

IMAGE_COUNT_PER_RECORD = 123
if PATCH_SIZE <= 256:
    PATCH_PER_IMAGE = 16
    TPU_BATCH_SIZE_MULTIPLIER = 64
else:
    PATCH_PER_IMAGE = 4
    TPU_BATCH_SIZE_MULTIPLIER = 128

if USE_ELEVATION:
    N_BANDS = 5  # RGB, alpha (because we use masking), elevation
else:
    N_BANDS = 4
CLASS_COUNT = 9  # see in predict.py

EARLY_STOPPING_PATIENCE = 24

PATH_TO_DATA = GCS_DS_PATH
IMGS_DIR = PATH_TO_DATA + "/images"
ELEVATION_DIR = PATH_TO_DATA + "/elevation"
MASKS_DIR = PATH_TO_DATA + "/masks"
PATCHES_PATH = PATH_TO_DATA + '/patches/p{}'.format(PATCH_SIZE)
INPUT_PATH = PATCHES_PATH + '/{}'
OUTPUT_DIR = "output"
CONFUSION_MATRIX_DIR = OUTPUT_DIR + '/confusion_matrix'

CATEGORIES = {
    'woodland': 1,
    'water': 2,
    'agricultural': 3,
    'urban': 4,
    'grassland': 5,
    'railway': 6,
    'roads': 7,
    'buildings': 8,
}


        
        


# data_handling:
import math
import random
import sys

import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from numpy import ndarray
from rasterio import DatasetReader
from rasterio.enums import Resampling
import tensorflow_addons as tfa

DOWNSCALING_RATE = 1.

ORIGINAL_IMAGE_PATCH_KEY = 'original'
ELEVATION_KEY = 'normalized_elevation'
GT_KEY = 'gt'
TFREC_FORMAT = {
    ORIGINAL_IMAGE_PATCH_KEY: tf.io.FixedLenFeature([], tf.string),  # tf.string means bytestring
    ELEVATION_KEY: tf.io.FixedLenFeature([], tf.string),  # tf.string means bytestring
    GT_KEY: tf.io.FixedLenFeature([], tf.string),   # shape [] means single element
}


# we can read bigger images one band at a time
def read_one_band(
    rasterio_reader: DatasetReader,
    band_id,
    crop_to=sys.maxsize,
    scale_factor=1. / DOWNSCALING_RATE
):
    # resample data to target shape
    data = rasterio_reader.read(
        band_id,
        out_shape=(
            1,
            int(rasterio_reader.height * scale_factor),
            int(rasterio_reader.width * scale_factor)
        ),
        resampling=Resampling.mode
    )
    return np.array(data[:crop_to, :crop_to])


def from_one_hot_to_category_indices(data: ndarray):
    """ converts data (numpy array) that has one-hot encoded categories in its last dimension
        to category indices (with the other dimensions remaining) """
    return np.argmax(data, axis=-1)


def read_image(img_path):
    return np.array(cv2.imread(img_path), dtype=np.uint8)


def read_mask(mask_path):
    return np.array(cv2.imread(mask_path), dtype=np.uint8)[:, :, 0]


def normalize(img):
    return np.array(img, dtype=np.float32) / 255.


# with per-image normalization
def read_elevation(elevation_path):
    e = np.array(Image.open(elevation_path))
    min_elevation = np.min(e)
    e = (e - min_elevation) / (np.max(e) - min_elevation)
    return e


def segmentation_map_from_mask(categories):
    image = np.zeros((categories.shape[0], categories.shape[1], 3), dtype=np.float32)
    for i in range(categories.shape[0]):
        for j in range(categories.shape[1]):
            for k in range(3):
                image[i, j, k] = CATEGORY_COLORS[categories[i, j]][k]
    image = image.transpose([2, 0, 1])
    return categories, normalize(image)


def add_channel(channel, original_image):
    return np.append(original_image, [channel], axis=0)


def add_masked_alpha_channel(mask, original_image):
    alpha = np.zeros(mask.shape, dtype=np.float32)
    alpha[np.nonzero(mask)] = 1.
    return add_channel(alpha, original_image)


def add_channel_tf(channel, original_image):
    return tf.concat([original_image, channel], axis=-1)


def add_masked_alpha_channel_tf(ground_truth, original_image):
    alpha = tf.where(tf.not_equal(ground_truth, tf.constant(0.)), ones, zeros)
    return add_channel_tf(alpha, original_image)


def convert_to_example(original_patch, elevation, gt):
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                ORIGINAL_IMAGE_PATCH_KEY: bytes_feature(tf.io.serialize_tensor(original_patch)),
                ELEVATION_KEY: bytes_feature(tf.io.serialize_tensor(elevation)),
                GT_KEY: bytes_feature(tf.io.serialize_tensor(gt))
            }
        )
    )


def parse_example(example_proto):
    # Parse the input tf.Example proto using the dictionary TFREC_FORMAT
    features = tf.io.parse_single_example(example_proto, TFREC_FORMAT)

    ground_truth = features[GT_KEY]
    ground_truth = tf.io.parse_tensor(ground_truth, np.uint8)
    ground_truth = tf.reshape(ground_truth, GROUND_TRUTH_SHAPE)
    mask = tf.cast(ground_truth, dtype=tf.float32)

    original_image_patch = features[ORIGINAL_IMAGE_PATCH_KEY]
    original_image_patch = tf.cast(
        tf.io.parse_tensor(original_image_patch, np.uint8),
        dtype=tf.float32)

    elevation_patch = features[ELEVATION_KEY]
    elevation_patch = tf.io.parse_tensor(elevation_patch, np.float32)
    elevation_patch = tf.reshape(elevation_patch, GROUND_TRUTH_SHAPE)

    if USE_ELEVATION:
        original_image_patch = add_channel_tf(elevation_patch, original_image_patch)
    original_image_patch = add_masked_alpha_channel_tf(mask, original_image_patch)
    original_image_patch = tf.reshape(original_image_patch, [PATCH_SIZE, PATCH_SIZE, N_BANDS])

    ground_truth = tf.one_hot(ground_truth, CLASS_COUNT, dtype=tf.float32)
    ground_truth = tf.reshape(ground_truth, [PATCH_SIZE, PATCH_SIZE, CLASS_COUNT])

    return original_image_patch, ground_truth


def parse_example_factory_with_augmentation(crop_boxes):
    global zeros, ones
    zeros = tf.zeros(GROUND_TRUTH_SHAPE, dtype=tf.float32)
    ones = tf.ones(GROUND_TRUTH_SHAPE, dtype=tf.float32)

    def inner_parse_example(example_proto):
        patch_img, patch_mask = parse_example(example_proto)
        return augment(patch_img, patch_mask, crop_boxes)

    return inner_parse_example


def parse_example_factory():
    global zeros, ones
    zeros = tf.zeros(GROUND_TRUTH_SHAPE, dtype=tf.float32)
    ones = tf.ones(GROUND_TRUTH_SHAPE, dtype=tf.float32)

    def inner_parse_example(example_proto):
        return parse_example(example_proto)

    return inner_parse_example


def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.numpy()]))


def transform_both(op, patch_img, patch_mask, args={}):
    return op(patch_img, **args), op(patch_mask, **args)


def augment(patch_img, patch_mask, crop_boxes):
    patch_img, patch_mask = random_rotate(patch_img, patch_mask)
    patch_img, patch_mask = random_zoom_in(patch_img, patch_mask, crop_boxes)
    return tf.squeeze(patch_img), tf.squeeze(patch_mask)


def random_rotate(patch_img, patch_mask):
    random_angle = tf.random.uniform([], 0, 1, dtype=tf.float32)
    patch_img, patch_mask = transform_both(tfa.image.rotate,  # fill mode: put zeros (masked out)
       patch_img, patch_mask,
       {'angles': random_angle * math.pi, 'interpolation': 'nearest'})
    return patch_img, patch_mask


def random_zoom_in(patch_img, patch_mask, crop_boxes):
    random_zoom_ind = tf.random.uniform([], 0, len(crop_boxes), dtype=tf.int32)
    random_crop_box = [tf.constant(np.array(crop_boxes), dtype=tf.float32)[random_zoom_ind]]

    patch_img = tf.image.crop_and_resize(
        tf.expand_dims(patch_img, 0),  # needs 4D tensor
        random_crop_box,
        box_indices=tf.zeros(1, dtype=tf.int32),
        crop_size=(PATCH_SIZE, PATCH_SIZE))
    patch_mask = tf.image.crop_and_resize(
        tf.expand_dims(patch_mask, 0),
        random_crop_box,
        box_indices=tf.zeros(1, dtype=tf.int32),
        crop_size=(PATCH_SIZE, PATCH_SIZE),
        method='nearest')  # we don't want to introduce invalid labels with interpolated values
    return patch_img, patch_mask







from typing import Callable, Union

import numpy as np
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.losses import binary_crossentropy


def tanimoto_loss() -> Callable[[tf.Tensor, tf.Tensor], tf.Tensor]:
    """
    Tanimoto loss. Defined in the paper "ResUNet-a: a deep learning framework for
    semantic segmentation of remotely sensed data", under 3.2.4. Generalization to multiclass
    imbalanced problems. See https://arxiv.org/pdf/1904.00592.pdf Used as loss function for
    multi-class image segmentation with one-hot encoded masks.

    :return: Tanimoto  loss function (Callable[[tf.Tensor, tf.Tensor], tf.Tensor])
    """

    def loss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        """
        Compute Tanimoto loss.
        :param y_true: True masks (tf.Tensor,
            shape=(<BATCH_SIZE>, <IMAGE_HEIGHT>, <IMAGE_WIDTH>, <N_CLASSES>))
        :param y_pred:
            Predicted masks (tf.Tensor, shape=(<BATCH_SIZE>, <IMAGE_HEIGHT>, <IMAGE_WIDTH>,
        <N_CLASSES>))
        :return: Tanimoto loss (tf.Tensor, shape=(None, ))
        """
        axis_to_reduce = range(1, K.ndim(y_pred))  # All axis but first (batch)
        numerator = y_true * y_pred
        numerator = K.sum(numerator, axis=axis_to_reduce)

        denominator = (y_true**2 + y_pred**2 - y_true * y_pred)
        denominator = K.sum(denominator, axis=axis_to_reduce)
        return 1 - numerator / denominator

    return loss


def tanimoto_with_complements() -> Callable[[tf.Tensor, tf.Tensor], tf.Tensor]:
    """
    From "ResUNet-a: a deep learning framework for semantic segmentation of remotely sensed data"
    """
    tanimoto = tanimoto_loss()

    def loss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        normal = tanimoto(y_true, y_pred)
        complement = tanimoto(1. - y_true, 1. - y_pred)
        return (normal + complement) / 2

    return loss





from tensorflow.python.keras import Model
from tensorflow.python.keras.callbacks import Callback


class SaveModelCallback(Callback):

    def __init__(self, model: Model, tf_checkpoint_path: str, pickle_path: str = None,
                 starting_val_loss=None):
        super().__init__()
        self.model = model
        self.tf_checkpoint_path = tf_checkpoint_path
        if pickle_path is not None:
            self.pickle_path = pickle_path
        else:
            self.pickle_path = tf_checkpoint_path + '_pickle'
        self.best_val_loss = starting_val_loss
        self.save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')

    def on_epoch_end(self, epoch, logs=None):
        self.model.save_weights(self.tf_checkpoint_path + '_last', options=self.save_locally, )
        if self.best_val_loss is None or logs['val_loss'] < self.best_val_loss:
            print('saving model')
            self.best_val_loss = logs['val_loss']
            self.model.save_weights(self.tf_checkpoint_path, options=self.save_locally)
            # pickle_file = open(self.pickle_path, 'wb')
            # pickle.dump(self.model, pickle_file)

            
            
            
            
            
            
            


# training

from keras_unet_collection.model_resunet_a_2d import resunet_a_2d
import tensorflow as tf
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, \
    TensorBoard, CSVLogger
from functools import reduce

from geonrw_unet_model import unet_model

patch_size_text = '{}'.format(PATCH_SIZE)
resunet_a_weights_path = 'weights_resunet_a_p' + patch_size_text
unet_weights_path = 'weights_unet_p' + patch_size_text
if not os.path.exists(resunet_a_weights_path):
    os.makedirs(resunet_a_weights_path)
if not os.path.exists(unet_weights_path):
    os.makedirs(unet_weights_path)
resunet_a_weights_path += '/saved_model'
unet_weights_path += '/saved_model'

resunet_a_saved_weights_path = RESUNETA_SAVED_PATH + '/weights_resunet_a_p256/saved_model'
unet_saved_weights_path = ''


def compile_model(model):
    # tf.python.keras.optimizer_v2.adam.Adam(learning_rate=x)   or 'adam' if that doesn't work
    model.compile(optimizer='adam', loss=tanimoto_with_complements())
    return model


def get_model_unet():
    model = unet_model(CLASS_COUNT, PATCH_SIZE, n_channels=N_BANDS, upconv=True)
    compile_model(model)
    return model


def get_model_resunet_a(batch_size):
    batch_norm = batch_size >= 2
    model = resunet_a_2d((PATCH_SIZE, PATCH_SIZE, N_BANDS), [32, 64, 128, 256, 512, 1024],
                         dilation_num=[1, 3, 15, 31],
                         n_labels=CLASS_COUNT,
                         activation='ReLU', output_activation='Sigmoid',
                         batch_norm=batch_norm, unpool=True, name='resunet')
    compile_model(model)
    return model


def make_crop_boxes_for_random_zoom_in():
    scales = list(np.arange(0.52, 1.0, 0.04))
    variation_count = 3
    crop_boxes = []  # will contain not centered boxes too
    for scale in scales:
        for j in range(variation_count):
            for k in range(variation_count):
                zoom_in_amount = 1. - scale
                x1 = 0. + j * (zoom_in_amount / 2.)
                y1 = 0. + k * (zoom_in_amount / 2.)
                x2 = x1 + scale
                y2 = y1 + scale
                crop_boxes.append([x1, y1, x2, y2])
    return crop_boxes


def count_available_inputs(tfrecord_paths: [str]):
    add_inputs_count_from_path = lambda i, path: i + int(path.split('_')[1])
    return reduce(add_inputs_count_from_path, tfrecord_paths, 0)


def setup_input_stream(
    usage_kind: str,
    batch_size=BASIC_BATCH_SIZE,
    data_divider=DATA_DIVIDER
):
    input_path = INPUT_PATH.format(usage_kind)
    
    tfrecord_paths = tf.io.gfile.glob(input_path + '/*.tfrec')
    inputs_used = int(count_available_inputs(tfrecord_paths) / DATA_DIVIDER)
    print('{} inputs used: {}'.format(usage_kind, inputs_used))

    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False

    if usage_kind == 'train':
        crop_boxes = make_crop_boxes_for_random_zoom_in()
        parser = parse_example_factory_with_augmentation(crop_boxes)
    else:
        parser = parse_example_factory()
    dataset = tf.data.TFRecordDataset(tfrecord_paths, num_parallel_reads=tf.data.AUTOTUNE)\
        .with_options(ignore_order)\
        .take(inputs_used)\
        .map(parser, num_parallel_calls=tf.data.AUTOTUNE)\
        .repeat()\
        .batch(batch_size, drop_remainder=True)\
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    if usage_kind == 'train':
        dataset = dataset.shuffle(buffer_size=50, reshuffle_each_iteration=True)
    return dataset, inputs_used


def train_net(batch_size, is_unet):
    if is_unet:
        weights_path = unet_weights_path
        saved_weights_path = unet_saved_weights_path
        batch_size = batch_size * UNET_BATCH_MULTIPLIER
        model = get_model_unet(batch_size)
        model_title = 'U-Net'
    else:  # ResUNet-a
        weights_path = resunet_a_weights_path
        saved_weights_path = resunet_a_saved_weights_path
        model = get_model_resunet_a(batch_size)
        model_title = 'ResUNet-a'
        
    train_inputs, train_inputs_used = setup_input_stream('train', batch_size)
    val_inputs, val_inputs_used = setup_input_stream('val', batch_size)

    train_steps_per_epoch = int(train_inputs_used / batch_size)
    print("Starting training {}, batch size: {}, training steps count: {}"
          .format(model_title, batch_size, train_steps_per_epoch))

    if CONTINUE_LEARNING:
        print('LOADING WEIGHTS')
        model.load_weights(saved_weights_path + 'last')

    callbacks = assemble_callbacks(weights_path, model)

    model.fit(train_inputs,
              batch_size=batch_size,
              epochs=N_EPOCHS,
              verbose=1,
              shuffle=True,
              callbacks=callbacks,
              validation_data=val_inputs,
              steps_per_epoch=train_steps_per_epoch,
              validation_steps=int(val_inputs_used / batch_size),
              )
    return model


def assemble_callbacks(weights_path, model):
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0,
                                   patience=EARLY_STOPPING_PATIENCE, verbose=1, mode='auto')
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.33, patience=5)
    model_checkpoint = SaveModelCallback(model, weights_path)
    csv_logger = CSVLogger('log_unet.csv', append=True, separator=';')
    callbacks = [model_checkpoint, csv_logger, reduce_lr, early_stopping]
    return callbacks


def run_train():
    try:  # detect TPUs
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()  # TPU detection
        strategy = tf.distribute.TPUStrategy(tpu)
        batch_size = BASIC_BATCH_SIZE * TPU_BATCH_SIZE_MULTIPLIER
    except ValueError:  # detect GPUs
        strategy = tf.distribute.get_strategy()  # default strategy that works on CPU and single GPU
        batch_size = BASIC_BATCH_SIZE

    with strategy.scope():
        train_net(batch_size, is_unet=False)
        #train_net(batch_size, is_unet=True)

In [18]:
run_train()

Received dilation rates: [1, 3, 15, 31]
Expanding dilation rates:
	depth-0, dilation_rate = [1, 3, 15, 31]
	depth-1, dilation_rate = [1, 3, 15, 31]
	depth-2, dilation_rate = [1, 3, 15]
	depth-3, dilation_rate = [1, 3, 15]
	depth-4, dilation_rate = [1]
	depth-5, dilation_rate = [1]
train inputs used: 54114
val inputs used: 5472
Starting training ResUNet-a, batch size: 2, training steps count: 27057
LOADING WEIGHTS
Epoch 1/250
  402/27057 [..............................] - ETA: 2:50:17 - loss: 0.3365

KeyboardInterrupt: 

## Evaluate

In [ ]:
import os
from datetime import datetime

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.python.keras import backend as K

USE_UNET = False
RUN_ON_CPU = False
EVALUATION_BATCH_SIZE = int(128 / BASIC_BATCH_SIZE)
if USE_UNET:
    EVALUATION_BATCH_SIZE *= UNET_BATCH_MULTIPLIER


class Evaluator:
    def __init__(self, is_unet=True):
        if is_unet:
            self.model = get_model_unet()
            self.model.load_weights(unet_weights_path)
            self.batch_size = BASIC_BATCH_SIZE * UNET_BATCH_MULTIPLIER
        else:
            self.model = get_model_resunet_a(BASIC_BATCH_SIZE)
            self.model.load_weights(resunet_a_saved_weights_path)
            self.batch_size = BASIC_BATCH_SIZE
        print('loading finished')
        self.test_sequence = setup_input_stream('test', self.batch_size).as_numpy_iterator()
        self.list_of_masked_ground_truth_categories = []
        self.masked_predictions = []
        self.confusion_matrices = []
        self.accuracies = []
        self.test_iteration_count = \
            int(MAX_VAL_STEPS_PER_EPOCH / EVALUATION_BATCH_SIZE / DATA_DIVIDER)

    def run(self):
        # go through the test_sequence and compute the metrics
        # computing confusion matrix requires a lot of RAM, that is why we need to do a number of
        #    iterations of the main loop instead of just one loop. If we would have the
        #    EVALUATION_BATCH_SIZE a small number it could happen that there would be some
        #    category of which we would not see in any test image, and it would ruin our final
        #    confusion matrix, it would not have percentages as we'd like, because when a category
        #    is not seen in an iteration, it would have 0s in its row in the conf matrix.
        for k in range(self.test_iteration_count):
            for i in range(EVALUATION_BATCH_SIZE):
                # both will have self.batch_size length of tensors
                image, ground_truth = next(self.test_sequence)
                # self.batch_size number of predictions
                prediction = self.model.predict(x=image, batch_size=self.batch_size)
                prediction = from_one_hot_to_category_indices(prediction)

                ground_truth_categories = from_one_hot_to_category_indices(ground_truth)
                ground_truth_categories_masked = \
                    ground_truth_categories[np.nonzero(ground_truth_categories)]
                masked_prediction = prediction[np.nonzero(ground_truth_categories)]
                if len(masked_prediction) == PATCH_SIZE * PATCH_SIZE * self.batch_size:
                    self.list_of_masked_ground_truth_categories.append(
                        ground_truth_categories_masked)
                    self.masked_predictions.append(masked_prediction)

                    self.compute_accuracy(ground_truth_categories_masked, i, masked_prediction, k)
            self.compute_confusion_matrix()  # needs a lot of RAM!
            self.masked_predictions = []
            self.list_of_masked_ground_truth_categories = []
        self.display_results()

    def display_results(self):
        average_acc = float(np.mean(self.accuracies, axis=0))
        print('average accuracy: {:.3f}'.format(average_acc))
        confusion_matrix = np.mean(self.confusion_matrices, axis=0)
        labels = list(CATEGORIES.keys())
        for i in range(CLASS_COUNT - 1):
            print('{}: {:.3f}'.format(labels[i], float(confusion_matrix[i, i])))
        fig = plt.figure(figsize=(10, 9))
        ax = fig.add_subplot(1, 1, 1)
        ConfusionMatrixDisplay(confusion_matrix, labels)\
            .plot(ax=ax, xticks_rotation=45., cmap='YlOrRd', values_format='.3f')
        file_name = '/cm_{date:%Y-%m-%d_%H:%M}_avg{avg}.png'\
            .format(date=datetime.now(), avg=round(average_acc))
        fig.savefig(CONFUSION_MATRIX_DIR + file_name)
        plt.show()

    def compute_confusion_matrix(self):
        #if len(self.list_of_masked_ground_truth_categories) == EVALUATION_BATCH_SIZE:
        confusion_matrix = metrics.confusion_matrix(
            np.array(self.list_of_masked_ground_truth_categories).flatten(),
            np.array(self.masked_predictions).flatten(),
            labels=range(1, CLASS_COUNT),
            normalize='true'
        ) * 100
        self.confusion_matrices.append(confusion_matrix)

    def compute_accuracy(self, masked_ground_truth_categories, i, masked_prediction,
                         test_iteration_index):
        total_pixel_count = masked_ground_truth_categories.size
        correct_pixel_count_unet = np.count_nonzero(
            masked_prediction == masked_ground_truth_categories
        )
        accuracy = correct_pixel_count_unet / total_pixel_count * 100
        self.accuracies.append(accuracy)
        print('{}      ({}/{})'.format(
            accuracy,
            i + test_iteration_index * EVALUATION_BATCH_SIZE + 1,
            int(self.test_iteration_count * EVALUATION_BATCH_SIZE)
        ))

        
def run_eval():
    if not os.path.exists(CONFUSION_MATRIX_DIR):
        os.makedirs(CONFUSION_MATRIX_DIR)
    # if we are training on GPU we might not have enough RAM on it to run this too,
    #   in this case we should use the CPU, otherwise using the GPU is faster
    if RUN_ON_CPU:
        config = tf.compat.v1.ConfigProto(device_count={'GPU': 0})
        sess = tf.compat.v1.Session(config=config)
        with tf.Graph().as_default():
            with sess:
                K.set_session(sess)
                Evaluator(is_unet=USE_UNET).run()
    else:  # run on GPU
        Evaluator(is_unet=USE_UNET).run()

In [ ]:
run_eval()

In [ ]:
# maybe we'll need it to get the outputs
#import shutil shutil.make_archive( zip_file_name , 'zip', folder_to_be_zipped)